# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [75]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [76]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print(df.head())
df.info()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 0 to 399
Data columns (total 4 columns):
admit       397 non-null int64
gre         397 non-null float64
gpa         397 non-null float64
prestige    397 non-null float64
dtypes: float64(3), int64(1)
memory usage: 15.5 KB


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [77]:
# frequency table for prestige and whether or not someone was admitted
number_admitted_prestige = df.groupby(['prestige']).admit.count()
scores_prestige = df.groupby(['prestige','admit']).count()
print("number admitted:")
print(number_admitted_prestige)
print("average scores:")
print(scores_prestige)

number admitted:
prestige
1.0     61
2.0    148
3.0    121
4.0     67
Name: admit, dtype: int64
average scores:
                gre  gpa
prestige admit          
1.0      0       28   28
         1       33   33
2.0      0       95   95
         1       53   53
3.0      0       93   93
         1       28   28
4.0      0       55   55
         1       12   12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [78]:
dummy = pd.get_dummies(df['prestige'], prefix = 'prest')
dummy

prest_1.0  prest_2.0  prest_3.0  prest_4.0
0            0          0          1          0
1            0          0          1          0
2            1          0          0          0
3            0          0          0          1
4            0          0          0          1
5            0          1          0          0
6            1          0          0          0
7            0          1          0          0
8            0          0          1          0
9            0          1          0          0
10           0          0          0          1
11           1          0          0          0
12           1          0          0          0
13           0          1          0          0
14           1          0          0          0
15           0          0          1          0
16           0          0          0          1
17           0          0          1          0
18           0          1          0          0
19           1          0          0          0
20           0          0          1          0
21           0          1          0          0
22           0          0          0          1
23           0          0          0          1
24           0          1          0          0
25           1          0          0          0
26           1          0          0          0
27           0          0          0          1
28           0          1          0          0
29           1          0          0          0
..         ...        ...        ...        ...
370          0          1          0          0
371          0          0          1          0
372          1          0          0          0
373          1          0          0          0
374          0          1          0          0
375          0          0          0          1
376          0          1          0          0
377          0          1          0          0
378          0          0          1          0
379          0          1          0          0
380          0          1          0          0
381          0          1          0          0
382          0          1          0          0
383          1          0          0          0
384          0          1          0          0
385          1          0          0          0
386          0          1          0          0
387          0          1          0          0
388          0          1          0          0
389          0          1          0          0
390          0          1          0          0
391          0          1          0          0
392          0          0          1          0
393          0          1          0          0
394          0          0          1          0
395          0          1          0          0
396          0          0          1          0
397          0          1          0          0
398          0          1          0          0
399          0          0          1          0

[397 rows x 4 columns]

In [79]:
df3 = df.join(dummy).drop('prestige', 1)
df3.head()

admit    gre   gpa  prest_1.0  prest_2.0  prest_3.0  prest_4.0
0      0  380.0  3.61          0          0          1          0
1      1  660.0  3.67          0          0          1          0
2      1  800.0  4.00          1          0          0          0
3      1  640.0  3.19          0          0          0          1
4      0  520.0  2.93          0          0          0          1

#### 2.2 When modeling our class variables, how many do we need? 



Answer:3 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [80]:
cols_to_keep = ['admit', 'gre', 'gpa']

In [81]:
handCalc = df[cols_to_keep].join(dummy.ix[:,'prest_1.0':])

In [82]:
handCalc.columns = ['admit', 'gre','gpa','prest_1','prest_2','prest_3','prest_4']
print(handCalc.head())
handCalc.info()

   admit    gre   gpa  prest_1  prest_2  prest_3  prest_4
0      0  380.0  3.61        0        0        1        0
1      1  660.0  3.67        0        0        1        0
2      1  800.0  4.00        1        0        0        0
3      1  640.0  3.19        0        0        0        1
4      0  520.0  2.93        0        0        0        1
<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 0 to 399
Data columns (total 7 columns):
admit      397 non-null int64
gre        397 non-null float64
gpa        397 non-null float64
prest_1    397 non-null uint8
prest_2    397 non-null uint8
prest_3    397 non-null uint8
prest_4    397 non-null uint8
dtypes: float64(2), int64(1), uint8(4)
memory usage: 14.0 KB


In [83]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
results = pd.crosstab(handCalc.admit, handCalc.prest_1)

yep_p1 = results.loc[1,1]
yep_p0 = results.loc[1,0]
nope = results.loc[0,0]
total_p1 = results.loc[0,1]+yep_p1
total_p0 = results.loc[0,0]+results.loc[1,0]

results

prest_1    0   1
admit           
0        243  28
1         93  33

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [84]:
first = yep_p1/total_p1
first

0.54098360655737709

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [85]:
not_first = yep_p0/total_p0
not_first

0.2767857142857143

#### 3.3 Calculate the odds ratio

In [86]:
first/not_first

1.9545214172395557

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of admission are higher for those attending a 1 ranked college

#### 3.5 Print the cross tab for prestige_4

In [87]:
cross_tab = pd.crosstab(handCalc.admit, handCalc.prest_4)
yes = cross_tab.loc[1,1]
total = cross_tab.loc[0,1]+yes
cross_tab

prest_4    0   1
admit           
0        216  55
1        114  12

#### 3.6 Calculate the OR 

In [88]:
yes/total

0.17910447761194029

#### 3.7 Write this finding in a sentence

Answer:The odds of prestige 4 leading to admit is low

## Part 4. Analysis

In [89]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
#data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
data = df[cols_to_keep].join(dummy.ix[:,'prest_2.0':])
data.columns = ['admit', 'gre','gpa','prest_2','prest_3','prest_4']
print(data.head())

   admit    gre   gpa  prest_2  prest_3  prest_4
0      0  380.0  3.61        0        1        0
1      1  660.0  3.67        0        1        0
2      1  800.0  4.00        0        0        0
3      1  640.0  3.19        0        0        1
4      0  520.0  2.93        0        0        1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [90]:
# manually add the intercept
data['intercept'] = 1.0

In [91]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()

#### 4.1 Set the covariates to a variable called train_cols

In [100]:
train_cols = data.columns[1:]
train_cols

Index(['gre', 'gpa', 'prest_2', 'prest_3', 'prest_4', 'intercept'], dtype='object')

#### 4.2 Fit the model

In [102]:
lm.fit(data[['gre']], data['admit'], data['gpa'])


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### 4.3 Print the summary results

In [101]:
coefficient = lm.coef_
intercept = lm.intercept_

print(coefficient, intercept)

[[ 0.00284543]] [-2.41969443]


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [104]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()
result
result.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 28 May 2017   Pseudo R-squ.:                 0.08166
Time:                        14:18:04   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prest_2       -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prest_3       -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prest_4       -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
==============================================================================
"""

In [105]:
np.exp(result.params)

gre          1.002221
gpa          2.180027
prest_2      0.506548
prest_3      0.262192
prest_4      0.211525
intercept    0.020716
dtype: float64

In [107]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print(np.exp(conf))

               2.5%     97.5%        OR
gre        1.000074  1.004372  1.002221
gpa        1.136120  4.183113  2.180027
prest_2    0.272168  0.942767  0.506548
prest_3    0.133377  0.515419  0.262192
prest_4    0.093329  0.479411  0.211525
intercept  0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: 50% chance with prestige 2

#### 4.6 Interpret the OR of GPA

Answer: 2.18 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [97]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [110]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print(gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print(gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


/Users/nataliebaer/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/nataliebaer/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:44: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


#### 5.1 Recreate the dummy variables

In [115]:
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prest_1', 'prest_2', 'prest_3', 'prest_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prest_2':])

/Users/nataliebaer/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/nataliebaer/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:44: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


#### 5.2 Make predictions on the enumerated dataset

In [125]:
combos['admit_pred'] = result.predict(combos[train_cols])
combos['admit_pred']


0      0.164173
1      0.090492
2      0.048977
3      0.039890
4      0.185907
5      0.103682
6      0.056492
7      0.046078
8      0.209795
9      0.118543
10     0.065080
11     0.053173
12     0.235865
13     0.135214
14     0.074871
15     0.061290
16     0.264090
17     0.153820
18     0.085999
19     0.070553
20     0.294393
21     0.174469
22     0.098605
23     0.081096
24     0.326629
25     0.197244
26     0.112831
27     0.093056
28     0.360591
29     0.222192
         ...   
370    0.201304
371    0.168977
372    0.527768
373    0.361479
374    0.226621
375    0.191201
376    0.565093
377    0.396929
378    0.254108
379    0.215590
380    0.601694
381    0.433495
382    0.283707
383    0.242159
384    0.637193
385    0.470800
386    0.315296
387    0.270871
388    0.671257
389    0.508433
390    0.348690
391    0.301632
392    0.703609
393    0.545972
394    0.383639
395    0.334286
396    0.734040
397    0.582995
398    0.419833
399    0.368608
Name: admit_pred, dtype:

#### 5.3 Interpret findings for the last 4 observations

In [127]:
combos.tail(4)

gre  gpa  prestige  intercept  prest_2  prest_3  prest_4  admit_pred
396  800.0  4.0       1.0        1.0        0        0        0    0.734040
397  800.0  4.0       2.0        1.0        1        0        0    0.582995
398  800.0  4.0       3.0        1.0        0        1        0    0.419833
399  800.0  4.0       4.0        1.0        0        0        1    0.368608

Answer: Even a perfect GRE and GPA does not guarantee admission

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.